In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, TensorDataset

import datetime
import os

import optuna
import torch.optim as optim

In [2]:
def save_loss_in_csv(model_num, loss):
    df = pd.read_csv('loss.csv')
    df = df.reindex(range(len(loss)))
    df[f"{model_num}"] = loss
    df.to_csv('loss.csv', index=False)

In [3]:
# build the gradient boosting regression model with PyTorch
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3= nn.Linear(hidden_size, hidden_size)
        self.fc4= nn.Linear(hidden_size, hidden_size)
        self.fc5= nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc3(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc4(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.fc5(out)
        return out
    
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self, outputs, labels):
        # Define your custom loss function here
        # This is a simple example, replace it with your own function
        custom_loss = 3*(torch.abs(labels - outputs))*(torch.abs(labels - 1/3) + torch.abs(labels -2/3))
        return custom_loss
    

    
def objective(trial):
    # Sample hyperparameters
    input_size = 8
    hidden_size = trial.suggest_int('hidden_size', 3, 1000, log=True)  # Single hidden size for all layers
    output_size = 1
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.3)

    # Instantiate the model with sampled hyperparameters
    model = Net(input_size, hidden_size, output_size, dropout_rate)

    # Define the loss function and optimizer
    criterion = CustomLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(100):
        training_loss = 0.0
        eval_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.float()
            labels = labels.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.reshape(-1)
            loss = criterion(outputs, labels).sum()
            loss.backward()
            optimizer.step()
            training_loss += loss.item()

        model.eval()
        outloss = 0
        # Iterate over the DataLoader for test data
        with torch.no_grad():
            for i, data in enumerate(test_loader):
                inputs, labels = data
                inputs = inputs.float()
                labels = labels.float()
                outputs = model(inputs).reshape(-1)
                loss = criterion(outputs, labels).sum()
                eval_loss += loss

        # Optuna logs the running loss for each epoch
        trial.report(eval_loss, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # Return the running loss as the objective value to minimize
    return eval_loss


class station_model():
    def __init__(self, station):
        self.station = station
        self.epoch_cnt = 20
        self.trial_cnt = 100

    def get_data_loader(self):
        df = pd.read_parquet(f'parquets/{self.station}.parquet')
        TOT = df['tot'].iloc[0]
        df['sbi'] = df['sbi']/TOT
        df['time'] = df['time']/1440

        # x is dataset without 'sbi', y is 'sbi'
        X = df.drop(['tot', 'sbi','bemp' ,'act', 'tot', 'station'], axis=1)
        y = df['sbi']

        X = X.to_numpy()
        y = y.to_numpy()

        X = torch.from_numpy(X)
        y = torch.from_numpy(y)
        dataset = TensorDataset(X, y)

        # get train, test loader
        self.all_data_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

        # split train, test
        train_size = int(0.8 * len(dataset))
        test_size = len(dataset) - train_size
        train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

        # get train, test loader
        self.train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)
        print(f'get data loader for {self.station}')

    def objective(self,trial):
        # Sample hyperparameters
        input_size = 8
        hidden_size = trial.suggest_int('hidden_size', 3, 2000, log=True)  # Single hidden size for all layers
        output_size = 1
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
        dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)

        # Instantiate the model with sampled hyperparameters
        model = Net(input_size, hidden_size, output_size, dropout_rate)

        # Define the loss function and optimizer
        criterion = CustomLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(self.epoch_cnt):
            training_loss = 0.0
            eval_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(self.train_loader):
                inputs, labels = data
                inputs = inputs.float()
                labels = labels.float()
                optimizer.zero_grad()
                outputs = model(inputs)
                outputs = outputs.reshape(-1)
                loss = criterion(outputs, labels).sum()
                loss.backward()
                optimizer.step()
                training_loss += loss.item()

            model.eval()
            # Iterate over the DataLoader for test data
            with torch.no_grad():
                for i, data in enumerate(self.test_loader):
                    inputs, labels = data
                    inputs = inputs.float()
                    labels = labels.float()
                    outputs = model(inputs).reshape(-1)
                    loss = criterion(outputs, labels).sum()
                    eval_loss += loss

            # Optuna logs the running loss for each epoch
            trial.report(eval_loss, epoch)
            # Handle pruning based on the intermediate value
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
        # Return the running loss as the objective value to minimize
        return eval_loss      


    def get_best_param(self):
        # Create an Optuna Study
        study = optuna.create_study(direction='minimize', storage='sqlite:///db.sqlite3', study_name=f'{self.station}', load_if_exists=True)
        num_trials = len(study.trials)

        study.optimize(self.objective, n_trials=self.trial_cnt)
        # Run the optimization process

        print(f'{self.station} eval_loss: {study.best_trial.value}')

        # Access the best hyperparameters
        best_params = study.best_trial.params

        self.best_params = best_params

        print(self.station,' Best hyperparameters:', best_params)

    def train(self):
        # Instantiate the final model with the best hyperparameters
        final_model = Net(8, self.best_params['hidden_size'], 1, self.best_params['dropout_rate'])

        # ... rest of your training code for the final model
        criterion = CustomLoss()
        optimizer = optim.Adam(final_model.parameters(), lr=self.best_params['learning_rate'])

        # train with whole dataset
        running_loss = 0.0
        loss_list = []
        for epoch in range(self.epoch_cnt):
            for i, data in enumerate(self.all_data_loader):
                inputs, labels = data
                inputs = inputs.float()
                labels = labels.float()
                optimizer.zero_grad()
                outputs = final_model(inputs)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels).sum()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()/128
            print(f'epoch: {epoch+1}, loss: {running_loss/self.all_data_loader.__len__()}')
            loss_list.append(running_loss/self.all_data_loader.__len__())
            running_loss = 0.0
        save_loss_in_csv(self.station, loss_list)
        
        final_model.eval()
        # save model
        torch.save(final_model, f'models/{self.station}')

In [6]:
available_station = np.loadtxt('html.2023.final.data/sno_test_set.txt', dtype='str')
# available_station = available_station[:35]
folder_path = 'models/'  # replace with your actual folder path
trained_models = os.listdir(folder_path)
available_station = np.setdiff1d(available_station, trained_models)
available_station = available_station[:35]
print(available_station)

['500101040' '500101041' '500101042' '500101091' '500101092' '500101093'
 '500101094' '500101114' '500101115' '500101123' '500101166' '500101175'
 '500101176' '500101181' '500101184' '500101185' '500101188' '500101189'
 '500101190' '500101191' '500101193' '500101199' '500101209' '500101216'
 '500101219' '500105066' '500106002' '500106003' '500106004' '500119043'
 '500119044' '500119045' '500119046' '500119047' '500119048']


In [5]:
for station_id in available_station:
    station = station_model(station_id)
    station.get_data_loader()
    station.get_best_param()
    station.train()
    print(f'{station_id} done')

FileNotFoundError: [Errno 2] No such file or directory: 'parquets/500101040.parquet'

In [8]:
# read loss.csv 
# train those station with loss avg > 0.35
loss_df = pd.read_csv('loss.csv')
# get station with loss avg > 0.35
station_list = []
for col in loss_df.columns:
    # if last element > 0.35, train this station
    if loss_df[col].iloc[-1] > 0.35:
        col = float(col)
        col = int(col)
        station_list.append(col)
print(station_list)

# train those station
for station_id in station_list:
    station = station_model(station_id)
    station.get_data_loader()
    station.get_best_param()
    station.train()
    print(f'train {station} done!')

[I 2023-12-16 19:49:42,506] Using an existing study with name '500101003' instead of creating a new one.


[500101003, 500101004, 500101008, 500101009, 500101010, 500101013, 500101014, 500101018, 500101026]
get data loader for 500101003


[I 2023-12-16 19:49:43,738] Trial 480 pruned. 
[I 2023-12-16 19:49:44,220] Trial 481 pruned. 
[I 2023-12-16 19:49:44,846] Trial 482 pruned. 
[I 2023-12-16 19:49:45,446] Trial 483 pruned. 
[I 2023-12-16 19:49:46,630] Trial 484 pruned. 
[I 2023-12-16 19:49:47,081] Trial 485 pruned. 
[I 2023-12-16 19:49:47,495] Trial 486 pruned. 
[I 2023-12-16 19:49:48,450] Trial 487 pruned. 
[I 2023-12-16 19:49:52,147] Trial 488 pruned. 
[I 2023-12-16 19:49:52,550] Trial 489 pruned. 
[I 2023-12-16 19:49:53,494] Trial 490 pruned. 
[I 2023-12-16 19:50:03,845] Trial 491 pruned. 
[I 2023-12-16 19:50:08,623] Trial 492 pruned. 
[I 2023-12-16 19:50:09,093] Trial 493 pruned. 
[I 2023-12-16 19:50:09,835] Trial 494 pruned. 
[I 2023-12-16 19:50:13,371] Trial 495 pruned. 
[I 2023-12-16 19:50:13,951] Trial 496 pruned. 
[I 2023-12-16 19:50:16,351] Trial 497 pruned. 
[I 2023-12-16 19:50:17,238] Trial 498 pruned. 
[I 2023-12-16 19:50:18,279] Trial 499 pruned. 
[I 2023-12-16 19:50:18,702] Trial 500 pruned. 
[I 2023-12-16

500101003 eval_loss: 5882.21875
500101003  Best hyperparameters: {'hidden_size': 151, 'learning_rate': 0.0012323738201189078, 'dropout_rate': 0.21124150624483878}
epoch: 1, loss: 0.4422799083621973
epoch: 2, loss: 0.3917129118428151
epoch: 3, loss: 0.38498967683692864
epoch: 4, loss: 0.3819094836532081
epoch: 5, loss: 0.37844545575108823
epoch: 6, loss: 0.37605048927336615
epoch: 7, loss: 0.3756995624097044
epoch: 8, loss: 0.37441762488442126
epoch: 9, loss: 0.3731591708474929
epoch: 10, loss: 0.37282656375908746
epoch: 11, loss: 0.3721342783078529
epoch: 12, loss: 0.37245812614309304
epoch: 13, loss: 0.37093430537264094
epoch: 14, loss: 0.37052025021156393
epoch: 15, loss: 0.37078560557255735
epoch: 16, loss: 0.36949656532900366
epoch: 17, loss: 0.3685296755705484
epoch: 18, loss: 0.36904708921999235
epoch: 19, loss: 0.36899127021184697


[I 2023-12-16 19:52:53,285] Using an existing study with name '500101004' instead of creating a new one.


...

500101018 eval_loss: 7757.27099609375
500101018  Best hyperparameters: {'hidden_size': 82, 'learning_rate': 0.0029400660933418497, 'dropout_rate': 0.45560509156989987}
epoch: 1, loss: 0.6850093115418925
epoch: 2, loss: 0.5929863170396149
epoch: 3, loss: 0.5777424343750668
epoch: 4, loss: 0.57404629137721
epoch: 5, loss: 0.5700123504051435
epoch: 6, loss: 0.5679855663582211
epoch: 7, loss: 0.5655943642329666
epoch: 8, loss: 0.5631518961797113
epoch: 9, loss: 0.562383515937296
epoch: 10, loss: 0.5617428755058962
epoch: 11, loss: 0.5581722158157628
epoch: 12, loss: 0.558258004025816
epoch: 13, loss: 0.5557566205121094
epoch: 14, loss: 0.5553607414300985
epoch: 15, loss: 0.5547742116999662
epoch: 16, loss: 0.5542949286835945
epoch: 17, loss: 0.5563584535100341
epoch: 18, loss: 0.5530253764281265
epoch: 19, loss: 0.5520224917348514


[I 2023-12-16 20:56:49,440] Using an existing study with name '500101026' instead of creating a new one.


epoch: 20, loss: 0.5522589449789009
train <__main__.station_model object at 0x17d7b1b90> done!
get data loader for 500101026


[I 2023-12-16 20:57:10,327] Trial 205 finished with value: 5896.7060546875 and parameters: {'hidden_size': 312, 'learning_rate': 0.0006529623101316045, 'dropout_rate': 0.26235623582136314}. Best is trial 205 with value: 5896.7060546875.
[I 2023-12-16 20:57:30,668] Trial 206 finished with value: 5977.5166015625 and parameters: {'hidden_size': 312, 'learning_rate': 0.0006087445879657995, 'dropout_rate': 0.2621260238078149}. Best is trial 205 with value: 5896.7060546875.
[I 2023-12-16 20:57:33,104] Trial 207 pruned. 
[I 2023-12-16 20:57:35,873] Trial 208 pruned. 
[I 2023-12-16 20:57:43,313] Trial 209 pruned. 
[I 2023-12-16 20:58:02,809] Trial 210 finished with value: 5994.60107421875 and parameters: {'hidden_size': 295, 'learning_rate': 0.0007618085831777872, 'dropout_rate': 0.25793649396277607}. Best is trial 205 with value: 5896.7060546875.
[I 2023-12-16 20:58:22,560] Trial 211 finished with value: 5911.04248046875 and parameters: {'hidden_size': 299, 'learning_rate': 0.0007253623141595

500101026 eval_loss: 5876.42431640625
500101026  Best hyperparameters: {'hidden_size': 296, 'learning_rate': 0.000790644409945961, 'dropout_rate': 0.25751223484625235}
epoch: 1, loss: 0.5025382466940139
epoch: 2, loss: 0.40982631053410323
epoch: 3, loss: 0.39145812098931404
epoch: 4, loss: 0.38460197876211744
epoch: 5, loss: 0.38022502319575435
epoch: 6, loss: 0.3792253107641617
epoch: 7, loss: 0.3751290423028609
epoch: 8, loss: 0.3747187431221814
epoch: 9, loss: 0.3744614759114533
epoch: 10, loss: 0.37257213951235985
epoch: 11, loss: 0.37267235285556155
epoch: 12, loss: 0.3728525435627855
epoch: 13, loss: 0.3715907512611933
epoch: 14, loss: 0.3705204709757507
epoch: 15, loss: 0.36951398065097374
epoch: 16, loss: 0.3706374197164452
epoch: 17, loss: 0.36892028267566973
epoch: 18, loss: 0.36983089296211846
epoch: 19, loss: 0.3698523592984874
epoch: 20, loss: 0.36788912998515194
train <__main__.station_model object at 0x17d608350> done!


In [9]:
df = pd.read_csv('loss.csv')
df

,500101003.0,500101004.0,500101005.0,500101006.0,500101007.0,500101008.0,500101009.0,500101010.0,500101013.0,500101014.0,...,500101010,500101013,500101014,500101018,500101019,500101024,500101026,500101031,500101037,500101039
0,0.444689,0.538689,0.331459,0.396226,0.372812,0.613214,0.561268,0.710761,0.525923,0.526660,...,0.606375,0.522900,0.520044,0.685009,0.392406,0.291617,0.502538,0.386127,0.339212,0.428888
1,0.395261,0.476579,0.316708,0.385969,0.350236,0.511558,0.501418,0.664368,0.500104,0.512044,...,0.485900,0.497036,0.491198,0.592986,0.372061,0.214250,0.409826,0.359999,0.304312,0.420668
2,0.387346,0.467370,0.313265,0.383831,0.347216,0.486738,0.486223,0.618513,0.484265,0.502888,...,0.452455,0.481752,0.471211,0.577742,0.365425,0.205954,0.391458,0.351646,0.292062,0.414385
3,0.382817,0.462352,0.311306,0.381893,0.346180,0.479111,0.476797,0.592754,0.473701,0.488586,...,0.443711,0.471202,0.462687,0.574046,0.362662,0.200216,0.384602,0.345904,0.284678,0.408163
4,0.378458,0.458872,0.310363,0.380193,0.344546,0.472084,0.472706,0.572710,0.468697,0.480242,...,0.436970,0.462013,0.458966,0.570012,0.360111,0.197163,0.380225,0.342101,0.281267,0.398697
5,0.377167,0.455809,0.308897,0.379075,0.343764,0.468867,0.466260,0.557047,0.463540,0.474583,...,0.434018,0.455000,0.455674,0.567986,0.357484,0.194032,0.379225,0.338014,0.277630,0.386378
6,0.375860,0.454142,0.308208,0.378053,0.342959,0.466838,0.464686,0.535636,0.457769,0.471129,...,0.432046,0.444856,0.453399,0.565594,0.355343,0.192943,0.375129,0.334768,0.276315,0.371611
7,0.375356,0.452423,0.307136,0.377295,0.342771,0.465613,0.465003,0.517334,0.453156,0.469102,...,0.428879,0.438376,0.451143,0.563152,0.353816,0.191459,0.374719,0.331472,0.276297,0.350956
8,0.373972,0.449503,0.305809,0.376403,0.342191,0.464973,0.463659,0.503593,0.447606,0.465231,...,0.428101,0.434227,0.449027,0.562384,0.351643,0.189875,0.374461,0.330133,0.274712,0.335674
9,0.373933,0.448108,0.304643,0.375276,0.342760,0.465499,0.463357,0.490896,0.439626,0.463448,...,0.425136,0.432196,0.449164,0.561743,0.350101,0.189323,0.372572,0.328008,0.273379,0.326247
